In [1]:
import sys
sys.path.append("..")
import nrc

In [2]:
lexicon = nrc.load_sentiments("../NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon.txt")

In [3]:
from numpy import array
from scipy.cluster.vq import vq, kmeans, whiten

In [4]:
ix = []
matrix = []
for k, v in lexicon.items():
    ix.append(k)
    matrix.append(v.vector)
matrix = array(matrix)

In [5]:
matrix

array([[-0.042,  0.212, -0.418],
       [ 0.04 ,  0.272, -0.436],
       [-0.146, -0.02 , -0.126],
       ...,
       [ 0.136, -0.306,  0.018],
       [-0.02 ,  0.04 , -0.076],
       [ 0.02 , -0.358, -0.5  ]])

In [6]:
#whitened = whiten(matrix)
clusters = kmeans(matrix, 5)

In [7]:
vq(matrix, clusters[0])

(array([4, 2, 2, ..., 1, 2, 1], dtype=int32),
 array([0.51103574, 0.60914212, 0.31185285, ..., 0.21693044, 0.20768218,
        0.33294386]))

In [8]:
lexicon["lie"].angle

148.69587650440903

In [9]:
lexicon["sly"].angle

68.98269912524374

In [10]:
lexicon["happy"].angle

25.173524524530166

In [11]:
cut = 10.0
from collections import defaultdict as ddict

In [12]:
vals = ddict(list)
sents = {}
for s, v in lexicon.items():
    vals[int(v.angle*cut)/cut].append(s)
    sents[s] = int(v.angle*cut)/cut
for s in sents:
    vals[sents[s]].sort(key=lambda x: (lexicon[x].valence**2+lexicon[x].arousal**2)**0.5)

In [13]:
def scale(x):
    return (lexicon[x].angle, vals[sents[x]])

In [14]:
scale("sad")

(-148.7308317576255, ['bounds', 'quarry', 'stereoscopic', 'sad'])

In [15]:
scale("happy")

(25.173524524530166,
 ['scorer', 'sympathize', 'debutante', 'recreational', 'happy'])

In [16]:
[str(lexicon[x]) for x in vals[0]]

['Sentiment: conglomerate(valence=0.0, arousal=0.0, dominance=0.6759999999999999)',
 'Sentiment: evidence(valence=0.0, arousal=0.0, dominance=0.556)',
 'Sentiment: factory(valence=0.0, arousal=0.0, dominance=0.3660000000000001)',
 'Sentiment: malleable(valence=0.0, arousal=0.0, dominance=-0.21799999999999997)',
 'Sentiment: microcosm(valence=0.0, arousal=0.0, dominance=0.040000000000000036)',
 'Sentiment: pong(valence=0.0, arousal=0.0, dominance=-0.20799999999999996)',
 'Sentiment: psychical(valence=0.0, arousal=0.0, dominance=0.6220000000000001)',
 'Sentiment: sidewinder(valence=0.0, arousal=0.0, dominance=-0.122)',
 'Sentiment: supernatant(valence=0.0, arousal=0.0, dominance=0.1299999999999999)',
 'Sentiment: vehement(valence=0.0, arousal=0.0, dominance=0.1499999999999999)',
 'Sentiment: wicket(valence=0.0, arousal=0.0, dominance=-0.05800000000000005)',
 'Sentiment: bagman(valence=0.01200000000000001, arousal=0.0, dominance=-0.21199999999999997)',
 'Sentiment: adjuvant(valence=0.0200

In [19]:
[str(lexicon[x]) for x in vals[180]]

['Sentiment: turret(valence=-0.01200000000000001, arousal=0.0, dominance=0.17599999999999993)',
 'Sentiment: budding(valence=-0.020000000000000018, arousal=0.0, dominance=-0.29000000000000004)',
 'Sentiment: encode(valence=-0.020000000000000018, arousal=0.0, dominance=0.46799999999999997)',
 'Sentiment: facsimile(valence=-0.020000000000000018, arousal=0.0, dominance=0.08600000000000008)',
 'Sentiment: interlocutory(valence=-0.020000000000000018, arousal=0.0, dominance=0.05400000000000005)',
 'Sentiment: pulsar(valence=-0.020000000000000018, arousal=0.0, dominance=0.236)',
 'Sentiment: rewrite(valence=-0.020000000000000018, arousal=0.0, dominance=-0.132)',
 'Sentiment: swank(valence=-0.020000000000000018, arousal=0.0, dominance=-0.11599999999999999)',
 'Sentiment: dissuade(valence=-0.03600000000000003, arousal=0.0, dominance=-0.30000000000000004)',
 'Sentiment: redo(valence=-0.038000000000000034, arousal=0.0, dominance=0.07000000000000006)',
 'Sentiment: barkeep(valence=-0.0400000000000

In [18]:
str(lexicon["shit"])

'Sentiment: shit(valence=-1.0, arousal=0.3560000000000001, dominance=-0.41200000000000003)'